<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [47]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [48]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "MaHGUoxMLg7uMCRr.sEdZufFXY5JpXwGOLdDX1KWixD5LrJ9YsekWlkVnaJcmkrbAmunjnzBabQyq6fOK"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1180822
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "China", "city": "Beijing", "street": "temple-of heaven, dongcheng", "aqicn_url": "https://api.waqi.info/feed/@447", "latitude": 40.19, "longitude": 116.41, "csv_file": "../../data/temple-of heaven, dongcheng-air-quality.csv"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [49]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Yesterday's Air Quality data (PM2.5) from the AQI API</span>


In [50]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,151.0,China,Beijing,"temple-of heaven, dongcheng",2024-11-20,https://api.waqi.info/feed/@447


In [51]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [52]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 40.25°N 116.5°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-20,2.00,0.0,4.610250,308.659821,Beijing
1,2024-11-21,2.45,0.0,6.120000,331.927612,Beijing
2,2024-11-22,2.35,0.0,3.219938,26.564985,Beijing
3,2024-11-23,1.05,0.0,2.902413,352.875061,Beijing
4,2024-11-24,4.45,0.0,1.938659,338.198517,Beijing
5,2024-11-25,-1.20,0.0,5.804825,277.124908,Beijing
6,2024-11-26,-2.05,0.0,1.800000,323.130005,Beijing
7,2024-11-27,-3.65,0.0,5.634891,243.435013,Beijing
8,2024-11-28,-1.10,0.0,4.693825,274.398621,Beijing
9,2024-11-29,3.45,0.0,9.360001,337.380096,Beijing


In [53]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data

# air_quality_fg.insert(aq_today_df,overwrite=False, operation="upsert")
# air_quality_df = air_quality_fg.read()##
# air_quality_df['pm25in3day'] = air_quality_df['pm25'].rolling(window=3).mean()##

# Get today's date
today = pd.Timestamp.now(tz='UTC').normalize()
yesterday = (today - pd.Timedelta(days=1)).normalize()
two_days_ago = (today - pd.Timedelta(days=2)).normalize()
three_days_ago = (today - pd.Timedelta(days=3)).normalize()

air_quality_df = air_quality_fg.read()
yesterday_data = air_quality_df[air_quality_df['date'] == yesterday]
two_days_ago_data = air_quality_df[air_quality_df['date'] == two_days_ago]
three_days_ago_data = air_quality_df[air_quality_df['date'] == three_days_ago]
avg = three_days_ago_data['pm25'].iloc[0]+yesterday_data['pm25'].iloc[0]+two_days_ago_data['pm25'].iloc[0]
avg = avg/3.0
aq_today_df['pm25in3day'] = avg   #EXTRA HERE
air_quality_fg.insert(aq_today_df,overwrite=True, operation="upsert")

air_quality_fg

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.24s) 
2024-11-20 16:28:28,433 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1180822/fs/1171525/fg/1357160


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1180822/jobs/named/air_quality_1_offline_fg_materialization/executions


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


In [ ]:
weather_fg.insert(daily_df,overwrite=True)


2024-11-20 16:28:37,774 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1180822/fs/1171525/fg/1357161


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1180822/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x27fb2b9b980>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 682011
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-20T03:28:37.000774Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwe

In [56]:

weather_df = weather_fg.read()


weather_df_sorted = weather_df.sort_values(by='date')


print(weather_df_sorted.tail(20)) 


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.22s) 
                          date  temperature_2m_mean  precipitation_sum  \
3579 2024-10-30 00:00:00+00:00            11.729248           0.000000   
1544 2024-10-31 00:00:00+00:00            11.364666           0.200000   
686  2024-11-01 00:00:00+00:00            13.260499           0.000000   
2506 2024-11-02 00:00:00+00:00            11.575084           0.000000   
748  2024-11-03 00:00:00+00:00            10.873000           0.000000   
3149 2024-11-04 00:00:00+00:00             4.612583           0.000000   
1197 2024-11-05 00:00:00+00:00             6.064667           0.000000   
2469 2024-11-06 00:00:00+00:00             6.498001           0.000000   
2221 2024-11-07 00:00:00+00:00             8.273000           0.000000   
2099 2024-11-08 00:00:00+00:00             9.085500           0.000000   
150  2024-11-09 00:00:00+00:00             9.937581           0.100000   
2402 2024-11-10 00:00:00+0

In [57]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pm25        1 non-null      float32       
 1   country     1 non-null      object        
 2   city        1 non-null      object        
 3   street      1 non-null      object        
 4   date        1 non-null      datetime64[ns]
 5   url         1 non-null      object        
 6   pm25in3day  1 non-null      float64       
dtypes: datetime64[ns](1), float32(1), float64(1), object(4)
memory usage: 184.0+ bytes
